## Preprocess Flight Delay Data
This only needs to be run once. It requires downloading all of the datasets from the Bureau of Transportation. To recreate, follow steps: 
- visit the [Bureau of Transportation](https://www.transtats.bts.gov/databases.asp?Z1qr_VQ=E&Z1qr_Qr5p=N8vn6v10&f7owrp6_VQF=D)
- select Airline Performance Data
- select download for Reporting Carrier On-Time Performance (1987-Present)
- select relevant features from GUI. See relevant_columns below
- download and unzip all files into Downloads folder (see src_filepath). This is set of 36 files. all 12 months per year for 2020, 2021, 2022

### Preprocess data steps
1. Trim data, keeping only relevant columns
2. Balance Data, keeping 10k samples from each month and class (Delayed, Not Delayed)
3. Join years and months and save to single csv file flight_data.csv

> |Total row count is 713,664. Slightly under 720k, as some months didn't have 10k samples for each class.

In [22]:
import pandas as pd


YEARS= ["2020", "2021", "2022"]
MONTHS =[ "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]
src_filepath = "~/Downloads/"
dest_filepath = "../data/"

RECORDS_TO_KEEP_FROM_EACH_DATASET = 5000

In [3]:
filename = "flight_data.csv"
flight_df = df = pd.read_csv(f"{src_filepath}/{filename}")

In [8]:
flight_df.columns

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
       'Reporting_Airline', 'Tail_Number', 'Flight_Number_Reporting_Airline',
       'Origin', 'Dest', 'DepTime', 'DepDelay', 'TaxiOut', 'WheelsOff',
       'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDel15',
       'Cancelled', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime',
       'AirTime', 'Flights', 'Distance', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'Carrier',
       'Full-time', 'Part-time', 'Grand Total'],
      dtype='object')

## Drop rows that are marked as diverted or cancelled

In [15]:
flight_df = flight_df[flight_df.Cancelled == 0]
flight_df = flight_df[flight_df.Diverted == 0]


In [17]:
# Drop columns that are not needed
flight_df = flight_df.drop(['Cancelled', 'Diverted', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay'], axis=1)

## For 36 months, grab 10k records from each month. (5k delayed, and 5k not delayed)

In [23]:
def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    # Find delayed flights, indicated by ArrDel15 == 1
    df_delayed = df[df["ArrDel15"] == 1]
    # Only calculating keep amount for delayed class, as its smaller
    keep_amount = min(len(df_delayed), RECORDS_TO_KEEP_FROM_EACH_DATASET)
    print(f"there are {len(df_delayed)} flight records that were delayed. randomly sampling {keep_amount}")
    df_delayed_sample = df_delayed.sample(n=keep_amount, random_state=1)

    df_not_delayed = df[df["ArrDel15"] == 0] 
    print(f"there are {len(df_not_delayed)} flight records that weren't delayed, sampling {keep_amount} of them")

    df_non_delayed_sample = df_not_delayed.sample(n=keep_amount, random_state=1)

    return pd.concat([df_delayed_sample, df_non_delayed_sample])


In [24]:
preprocessed_data = []
for year in YEARS:
    for month in MONTHS:
        print(f"processing year:{year} and month:{month}")
        preprocessed_data.append(preprocess_data(flight_df[(flight_df["Year"] == int(year)) & (flight_df["Month"] == int(month))]))
    
all_flights_df = pd.concat(preprocessed_data)

processing year:2020 and month:1
there are 82285 flight records that were delayed. randomly sampling 5000
there are 516983 flight records that weren't delayed, sampling 5000 of them
processing year:2020 and month:2
there are 84616 flight records that were delayed. randomly sampling 5000
there are 483460 flight records that weren't delayed, sampling 5000 of them
processing year:2020 and month:3
there are 53720 flight records that were delayed. randomly sampling 5000
there are 483524 flight records that weren't delayed, sampling 5000 of them
processing year:2020 and month:4
there are 9038 flight records that were delayed. randomly sampling 5000
there are 173968 flight records that weren't delayed, sampling 5000 of them
processing year:2020 and month:5
there are 7794 flight records that were delayed. randomly sampling 5000
there are 160853 flight records that weren't delayed, sampling 5000 of them
processing year:2020 and month:6
there are 14949 flight records that were delayed. randomly 

In [25]:
all_flights_df.describe()

,Year,Quarter,Month,DayofMonth,DayOfWeek,Flight_Number_Reporting_Airline,DepTime,DepDelay,TaxiOut,WheelsOff,...,ArrDelay,ArrDel15,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,Full-time,Part-time,Grand Total
count,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,...,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,360000.0,360000.000000,358754.000000,358754.000000,358754.000000
mean,2021.000000,2.500000,6.500000,15.733522,4.039153,2538.964169,1385.208219,28.711775,17.883931,1408.563928,...,26.670942,0.500000,142.023333,139.983686,114.126142,1.0,815.165475,40896.425107,3907.236535,44803.661643
std,0.816498,1.118036,3.452057,8.744654,2.009596,1778.124821,492.271123,74.593500,11.835681,494.764773,...,75.450179,0.500001,70.052962,71.527463,69.273166,0.0,576.230327,32754.907819,4377.865772,35962.942062
min,2020.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-62.000000,1.000000,1.000000,...,-82.000000,0.000000,10.000000,16.000000,8.000000,1.0,29.000000,2357.000000,0.000000,2374.000000
25%,2020.000000,1.750000,3.750000,8.000000,2.000000,1035.000000,1004.000000,-4.000000,11.000000,1020.000000,...,-11.000000,0.000000,90.000000,87.000000,63.000000,1.0,391.000000,10932.000000,740.000000,13143.000000
50%,2021.000000,2.500000,6.500000,16.000000,4.000000,2164.000000,1411.000000,2.000000,14.000000,1424.000000,...,14.500000,0.500000,127.000000,125.000000,99.000000,1.0,678.000000,53197.000000,1818.000000,54698.000000
75%,2022.000000,3.250000,9.250000,23.000000,6.000000,3902.000000,1804.000000,37.000000,20.000000,1820.000000,...,37.000000,1.000000,172.000000,172.000000,145.000000,1.0,1055.000000,65424.000000,5655.000000,71849.000000
max,2022.000000,4.000000,12.000000,31.000000,7.000000,8819.000000,2400.000000,2175.000000,218.000000,2400.000000,...,2186.000000,1.000000,695.000000,716.000000,677.000000,1.0,5812.000000,97373.000000,16424.000000,109108.000000


In [26]:
all_flights_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 360000 entries, 559890 to 17400394
Data columns (total 30 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Year                             360000 non-null  int64  
 1   Quarter                          360000 non-null  int64  
 2   Month                            360000 non-null  int64  
 3   DayofMonth                       360000 non-null  int64  
 4   DayOfWeek                        360000 non-null  int64  
 5   FlightDate                       360000 non-null  object 
 6   Reporting_Airline                360000 non-null  object 
 7   Tail_Number                      360000 non-null  object 
 8   Flight_Number_Reporting_Airline  360000 non-null  int64  
 9   Origin                           360000 non-null  object 
 10  Dest                             360000 non-null  object 
 11  DepTime                          360000 non-null  float64


In [27]:
output_file = "flight_data.csv"
print(f"writing to {output_file}")
all_flights_df.to_csv(dest_filepath + output_file, index=False)

writing to flight_data.csv
